In [ ]:
import bmll2 as b2
b2.get_file('modules/auxiliary_functions.py')

In [ ]:
import auxiliary_functions as af

import random
import math
import pandas as pd
import numpy as np
from pandas import StringDtype

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import LogFormatterSciNotation
from scipy.stats import linregress
from scipy.optimize import curve_fit
from scipy.stats import t

In [ ]:
b2.get_file('test_data/GFI_impact_profile.csv')
b2.get_file('test_data/GRT_impact_profile.csv')

impact_profile_GFI = pd.read_csv('GFI_impact_profile.csv')
impact_profile_GRT = pd.read_csv('GRT_impact_profile.csv')

In [ ]:
bins = np.linspace(0, 1, 51)
phi_bins_GFI = pd.cut(impact_profile_GFI['phi'], bins = bins)

dynamic_impact_bins_vals_GFI = (impact_profile_GFI['scaled impact']).groupby(phi_bins_GFI, observed = False).mean()
phi_bins_centers_GFI         = [interval.mid for interval in dynamic_impact_bins_vals_GFI.index.categories]

prepend = pd.Series([0], index = ['0'])
dynamic_impact_bins_vals_GFI = pd.concat([prepend, dynamic_impact_bins_vals_GFI])
phi_bins_centers_GFI.insert(0, 0)

bins = np.linspace(0, 1, 51)
phi_bins_GRT = pd.cut(impact_profile_GRT['phi'], bins = bins)

dynamic_impact_bins_vals_GRT = (impact_profile_GRT['scaled impact']).groupby(phi_bins_GRT, observed = False).mean()
phi_bins_centers_GRT         = [interval.mid for interval in dynamic_impact_bins_vals_GRT.index.categories]

prepend = pd.Series([0], index = ['0'])
dynamic_impact_bins_vals_GRT = pd.concat([prepend, dynamic_impact_bins_vals_GRT])
phi_bins_centers_GRT.insert(0, 0)


In [ ]:
def execution_profile_model(phi, exponent, I):
    return  I * (phi ** exponent)

phi    = np.array(phi_bins_centers_GRT)
impact = np.array(dynamic_impact_bins_vals_GRT)

mask       = phi >= 0 
phi_fit    = phi[mask]
impact_fit = impact[mask]

params, covariance = curve_fit(execution_profile_model, phi_fit, impact_fit, p0 = [0.5, 0.0002])

exponent_est_GRT = params[0]
I_est_GRT        = params[1]
print('Estimated exponent:', exponent_est_GRT)
print('Estimated peak impact', I_est_GRT)

fitted_curve_GRT = execution_profile_model(phi_bins_centers_GRT, exponent_est_GRT, I_est_GRT)

n   = len(phi_fit)
p   = len(params)           # number of parameters (2 here)
dof = n - p                # degrees of freedom

alpha = 0.05              # 95% CI
tval  = t.ppf(1 - alpha / 2, dof)
se    = np.sqrt(np.diag(covariance))
interval_half_widths = tval * se[0], tval * se[1]
print('Interval half widths:', interval_half_widths)

exponent_ci = exponent_est_GRT - tval * se[0], exponent_est_GRT + tval * se[0]
I_ci        = I_est_GRT - tval * se[1], I_est_GRT + tval * se[1]
print("Exponent 95% CI:", exponent_ci)
print("Scaling I 95% CI:", I_ci)

In [ ]:
log_x = np.log(np.array(phi_bins_centers_GRT)[1:])
log_y = np.log(np.array(dynamic_impact_bins_vals_GRT)[1:])

slope_GRT, intercept_GRT, r_value, p_value, std_err = linregress(log_x, log_y)
print(slope_GRT)
print(np.exp(intercept_GRT))

phi_vals = np.array(phi_bins_centers_GRT)
phi_vals = phi_vals[0:]

fitted_curve_GRT = np.exp(intercept_GRT) * phi_vals ** slope_GRT

In [ ]:
concave_line_GRT = 0.00042 * np.sqrt(phi_bins_centers_GRT)
straight_line_GRT = 0.00042 * np.array(phi_bins_centers_GRT)

fig, ax = plt.subplots(figsize = (6, 6))

#ax.set_title('Concave profile during metaorder execution(GRT)')
ax.scatter(phi_bins_centers_GRT, dynamic_impact_bins_vals_GRT, color = 'olive', marker = 'o', label = 'GRT dynamic impact')
ax.set_xlabel(r'Rescaled volume time $\phi = \sum q_i/Q$', fontsize = 14)
ax.set_ylabel(r'Dynamic impact $\frac{I(\phi Q)}{\sigma_{D} \sqrt{Q}}$', fontsize = 14)
#ax.plot(phi_bins_centers_GRT, concave_line_GRT, color = 'red', linestyle = '--', label = r'y = Y$\sqrt{\phi}$')
#ax.plot(phi_bins_centers_GRT, straight_line_GRT, color = 'black', linestyle = '--', label = r'y = Y$\phi$')
ax.plot(phi_bins_centers_GRT, fitted_curve_GRT, color = 'red', linestyle = '--', label = rf'Fitted curve: $Y \times\phi^{{{exponent_est_GRT:.2f}}}$')
ax.legend()

ax.set_box_aspect(1)
plt.tight_layout()

plt.savefig('concave_profile_GRT(20ADV)_2.pdf')
b2.put_file('concave_profile_GRT(20ADV)_2.pdf', 'figures')
b2.get_file('figures/concave_profile_GRT(20ADV)_2.pdf')

plt.show()

In [ ]:
def execution_profile_model(phi, exponent, I):
    return  I * (phi ** exponent)

phi    = np.array(phi_bins_centers_GFI)
impact = np.array(dynamic_impact_bins_vals_GFI)

mask       = phi >= 0 
phi_fit    = phi[mask]
impact_fit = impact[mask]

params, covariance = curve_fit(execution_profile_model, phi_fit, impact_fit, p0 = [0.5, 0.0002])

exponent_est_GFI = params[0]
I_est_GFI        = params[1]
print('Estimated exponent:', exponent_est_GFI)
print('Estimated peak impact', I_est_GFI)

fitted_curve_GFI = execution_profile_model(phi_bins_centers_GFI, exponent_est_GFI, I_est_GFI)

n   = len(phi_fit)
p   = len(params)           # number of parameters (2 here)
dof = n - p                # degrees of freedom

alpha = 0.05              # 95% CI
tval  = t.ppf(1 - alpha / 2, dof)
se    = np.sqrt(np.diag(covariance))
interval_half_widths = tval * se[0], tval * se[1]
print('Interval half widths:', interval_half_widths)

exponent_ci = exponent_est_GRT - tval * se[0], exponent_est_GRT + tval * se[0]
I_ci        = I_est_GRT - tval * se[1], I_est_GRT + tval * se[1]
print("Exponent 95% CI:", exponent_ci)
print("Scaling I 95% CI:", I_ci)

In [ ]:
log_x = np.log(np.array(phi_bins_centers_GFI)[1:])
log_y = np.log(np.array(dynamic_impact_bins_vals_GFI)[1:])

slope_GFI, intercept_GFI, r_value, p_value, std_err = linregress(log_x, log_y)
print(slope_GFI)
print(np.exp(intercept_GFI))

phi_vals = np.array(phi_bins_centers_GFI)
phi_vals = phi_vals[0:]

fitted_curve_GFI = np.exp(intercept_GFI) * phi_vals ** slope_GFI

In [ ]:
concave_line_GFI = 1.2 * np.sqrt(phi_bins_centers_GFI)
straight_line_GFI = 1.2 * np.array(phi_bins_centers_GFI)


fig, ax = plt.subplots(figsize = (6, 6))

#ax.set_title('Concave profile during metaorder execution(GFI)')
ax.scatter(phi_bins_centers_GFI, dynamic_impact_bins_vals_GFI, color = 'purple', marker = 'o', label = 'GFI dynamic impact')
ax.set_xlabel(r'Rescaled volume time $\phi = \sum q_i/Q$', fontsize = 14)
ax.set_ylabel(r'Dynamic impact $\frac{I(\phi Q)}{\sigma_{D} \sqrt{Q}}$', fontsize = 14)
#ax.plot(phi_bins_centers_GFI, concave_line_GFI, color = 'red', linestyle = '--', label = r'y = Y$\sqrt{\phi}$')
#ax.plot(phi_bins_centers_GFI, straight_line_GFI, color = 'black', linestyle = '--', label = r'y = Y$\phi$')
ax.plot(phi_bins_centers_GFI, fitted_curve_GFI, color = 'red', linestyle = '--', label =rf'Fitted curve: $Y\times\phi^{{{exponent_est_GFI:.2f}}}$')
ax.legend()

ax.set_box_aspect(1)
plt.tight_layout()

plt.savefig('concave_profile_GFI(20ADV)_2.pdf')
b2.put_file('concave_profile_GFI(20ADV)_2.pdf', 'figures')
b2.get_file('figures/concave_profile_GFI(20ADV)_2.pdf')

plt.show()

In [ ]:
execution_concavity_table = pd.DataFrame({'': ['GFI', 'GRT'], 'scaling factor': [I_est_GFI, I_est_GRT],
                                          'exponent': [exponent_est_GFI, exponent_est_GRT]})
print(execution_concavity_table.to_latex(index = False, caption = 'Fitted values for metaorder execution profiles of GFI and GRT',
                                         label = 'tab:execution profile values'))

In [ ]:
# Test on a single metaorder
test_metaorders = []

for date, day_D in GRT.groupby('Date', sort=True):
    trades = day_D.loc[day_D['Price'] != 0]
    N = 20
    f = af.trader_participation(N = N, method = 'power', alpha = 2, f_min = 1, 
                                f_max = trades.shape[0], seed = 1)
    c = af.cumulative_probs(f)
    
    if trades.empty:
        continue
        
    output = af.orders(N=N, trades=trades, cumulative_probs=c)
    
    for n in range(N):
        trader_n_trades = trades.iloc[output[n], ]
        if trader_n_trades.empty:
            continue
            
        trader_n_metaorders = af.metaorders(trader_n_trades)
        
        for i in range(len(trader_n_metaorders)):
            metaorder_i = trader_n_metaorders[i]
            
            if len(metaorder_i) >= 10:  # Find metaorders with 10+ child orders
                test_metaorders.append(metaorder_i)
                
            if len(test_metaorders) >= 20:  # Get 5 example metaorders
                break
        if len(test_metaorders) >= 20:
            break
    if len(test_metaorders) >= 20:
        break

print(len(test_metaorders))

In [ ]:
# check the plots
# Analyze the first test metaorder

metaorder_i = test_metaorders[10]

print(f"Metaorder has {len(metaorder_i)} child orders")
print(f"Sign: {metaorder_i['Trade Sign'].iloc[0]}")
print(f"Total volume: {metaorder_i['Volume'].sum()}")

# Calculate phi
phi = metaorder_i['Volume'].cumsum() / metaorder_i['Volume'].sum()

# Calculate impact
p_start = metaorder_i['Mid-price before'].iloc[0]
p_after = metaorder_i['Mid-price after(immediate)']
sign = metaorder_i['Trade Sign'].iloc[0]

print(f"\nStarting mid-price: {p_start}")
print(f"Ending mid-price: {p_after.iloc[-1]}")
print(f"Total price change: {p_after.iloc[-1] - p_start}")
print(f"Total log return: {np.log(p_after.iloc[-1] / p_start)}")
print(f"Exponent of Q/V: {np.log10(metaorder_i['Volume'].sum() / metaorder_i['Daily Volume'].iloc[0])}")

# Show the data
result_df = pd.DataFrame({
    'child order': np.array(range(len(metaorder_i))) + 1,
    'phi': phi.values,
    'mid price after': p_after.values,
    'log return from start': np.log(p_after / p_start).values * sign,
    'volume': metaorder_i['Volume'].values
})

print("\nData for each child order:")
print(result_df)

print(result_df.to_latex(index = False, caption = 'Example metaorder from GRT with 20 traders and a power-law trader participation distribution',
                                         label = 'tab:example metaorder'))
# Plot
#fig, axes = plt.subplots(1, 2, figsize=(15, 4))
# Plot 1: Price vs phi
#axes[3].plot(phi, p_after, 'o-')
#axes[3].axhline(p_start, color='r', linestyle='--', alpha=0.5, label='Start price')
#axes[3].set_xlabel('φ')
#axes[3].set_ylabel('Mid-price after')
#axes[3].set_title('Price evolution')
#axes[3].legend()
#axes[3].grid(True, alpha=0.3)

# Plot 2: Log return vs phi
log_return = np.log(p_after / p_start)
sign = metaorder_i['Trade Sign'].iloc[0]
signed_return = log_return * sign

fig, axes = plt.subplots(figsize = (6, 6))

axes.plot(phi, signed_return, 'o-', label='Actual')
# Theoretical sqrt curve
final_impact = signed_return.iloc[-1]
axes.plot(phi, np.sqrt(phi) * final_impact, 'r--', label = r'$\sqrt{\phi}I(Q)$')
axes.set_xlabel(r'Rescaled volume time $\phi = \sum q_i/Q$')
axes.set_ylabel(r'$I(\phi Q)$')
#axes[0].set_title('Impact evolution (should be concave)')
axes.legend()
axes.grid(True, alpha=0.3)

ax.set_box_aspect(1)
plt.tight_layout()

plt.savefig('single_concave_profile_GRT(20ADV).pdf')
b2.put_file('single_concave_profile_GRT(20ADV).pdf', 'figures')
b2.get_file('figures/concave_profile_GRT(20ADV).pdf')

plt.show()

# Plot 3: Normalized impact vs phi
Q = metaorder_i['Volume'].sum()
sigma_D = metaorder_i['Daily Volatility'].iloc[0]
volume_D = metaorder_i['Daily Volume'].iloc[0]
normalized_impact = signed_return / (sigma_D * np.sqrt(Q))

fig, axes = plt.subplots(figsize = (6, 6))

axes.plot(phi, normalized_impact, 'o-', label = 'Actual')
# Theoretical curve - should still be concave
Y = normalized_impact.iloc[-1]
axes.plot(phi, np.sqrt(phi) * Y, 'r--', label = r'$\sqrt{\phi} \times \left(\frac{I(Q)}{\sigma_{D} \sqrt{Q}}\right)$')
axes.set_xlabel(r'Rescaled volume time $\phi = \sum q_i/Q$')
axes.set_ylabel(r'Dynamic impact $\frac{I(\phi Q)}{\sigma_{D} \sqrt{Q}}$')
#axes[1].set_title('I(φQ) / (σ_D √Q)')
axes.legend()
axes.grid(True, alpha=0.3)

ax.set_box_aspect(1)
plt.tight_layout()

plt.savefig('single_concave_profile_GRT(20ADV)_scaled.pdf')
b2.put_file('single_concave_profile_GRT(20ADV)_scaled.pdf', 'figures')
b2.get_file('figures/single_concave_profile_GRT(20ADV)_scaled.pdf')

plt.show()
